In [1]:
import numpy as np
import pandas as pd
import 

In [ ]:
def minibatch_gradient_descent(train, test, initial_guess, batch_size, learning_rate, tolerance):
    n = 0
    w_n = initial_guess
    while 